In [1]:
import os
import torch
import torch.nn as nn
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.models import resnet18, resnet50, resnet101, resnet152
from tqdm import tqdm

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
def calc_acc(output, target):
    predicted = torch.max(output, 1)[1]
    num_samples = target.size(0)
    num_correct = (predicted == target).sum().item()
    return num_correct / num_samples

In [4]:
def validation(model, device, valid_loader, criterion):
    # ===============================
    # TODO 6: switch the model to validation mode
    model.eval()
    # ===============================
    valid_acc = 0.0
    valid_loss = 0.0

    # =========================================
    # TODO 7: turn off the gradient calculation
    with torch.set_grad_enabled(False):
    # =========================================
        for data, target in tqdm(valid_loader):
            data, target = data.to(device), target.to(device)

            output = model(data)

            # ================================
            # TODO 8: calculate accuracy, loss
            loss = criterion(output, target)
            valid_loss += loss.item()
            valid_acc += calc_acc(output, target)
            # ================================

    valid_acc /= len(valid_loader)
    valid_loss /= len(valid_loader)

    return valid_acc, valid_loss

In [5]:

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

Resize to 224*224

In [6]:
import os
import cv2
import numpy as np
# 原本切割好的檔案路徑
origin_path = '/content/drive/MyDrive/Colab Notebooks/project/training_224/test_cut'
# resize成224的檔案路徑
resized_path = '/content/drive/MyDrive/Colab Notebooks/project/training_224/resized_cut'
ext = '.png'
def cv_imread(filepath):
    cv_img = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), -1)
    # cv_img=cv2.cvtColor(cv_img, cv2.COLOR_RGB2BGR)
    return cv_img

print('start converting...')

# from_dir_path = os.path.join(origin_path, str(1))
to_dir_path = os.path.join(resized_path)
os.makedirs(to_dir_path, exist_ok=True)

pics = sorted(os.listdir(origin_path))
# print(pics)
# print(f'number[{i}] pictures = {len(pics)}')

for i, pic in enumerate(pics):
    print(f'i={i},pic={pic}')
    from_path = os.path.join(origin_path, pic)
    to_path = os.path.join(to_dir_path, f'{i}{ext}')

    image = cv_imread(from_path)
    image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
    cv2.imwrite(to_path, image)

start converting...
i=0,pic=Pic_1.png
i=1,pic=Pic_10.png
i=2,pic=Pic_11.png
i=3,pic=Pic_12.png
i=4,pic=Pic_13.png
i=5,pic=Pic_14.png
i=6,pic=Pic_15.png
i=7,pic=Pic_16.png
i=8,pic=Pic_17.png
i=9,pic=Pic_18.png
i=10,pic=Pic_19.png
i=11,pic=Pic_2.png
i=12,pic=Pic_20.png
i=13,pic=Pic_21.png
i=14,pic=Pic_22.png
i=15,pic=Pic_23.png
i=16,pic=Pic_24.png
i=17,pic=Pic_25.png
i=18,pic=Pic_26.png
i=19,pic=Pic_27.png
i=20,pic=Pic_28.png
i=21,pic=Pic_29.png
i=22,pic=Pic_3.png
i=23,pic=Pic_30.png
i=24,pic=Pic_31.png
i=25,pic=Pic_32.png
i=26,pic=Pic_33.png
i=27,pic=Pic_34.png
i=28,pic=Pic_35.png
i=29,pic=Pic_36.png
i=30,pic=Pic_37.png
i=31,pic=Pic_38.png
i=32,pic=Pic_39.png
i=33,pic=Pic_4.png
i=34,pic=Pic_40.png
i=35,pic=Pic_41.png
i=36,pic=Pic_42.png
i=37,pic=Pic_5.png
i=38,pic=Pic_6.png
i=39,pic=Pic_7.png
i=40,pic=Pic_8.png
i=41,pic=Pic_9.png


Set model & device

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = resnet18(pretrained=False)
model.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = nn.Linear(in_features=model.fc.in_features, out_features=1000, bias=True)
BATCH_SIZE=32

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Predict acc (可略)

In [8]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
# pt檔案
MODEL_PT = '/content/drive/MyDrive/Colab Notebooks/project/training_224/model/Resnet18_5.pt'
model.load_state_dict(torch.load(MODEL_PT, map_location=torch.device('cpu')))

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
dict_label = {}

LEARNING_RATE = 0.0001
BATCH_SIZE =32
EPOCHS = 100
# 有用資料夾標記label的resized224檔案
VALID_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/project/training_224/resized_224_2'

DOWNLOAD_MNIST = False


valid_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

valid_data = datasets.ImageFolder(VALID_DATA_PATH, transform=valid_transform)
valid_loader = DataLoader(valid_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# model = resnet.to(device).train()

criterion = nn.CrossEntropyLoss(ignore_index=-1)
# optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.98)

test_acc, test_loss = validation(model, device, valid_loader, criterion)
print(f'test_acc={test_acc*100}% test_loss={test_loss}')




Mounted at /content/drive


  0%|          | 0/1 [00:11<?, ?it/s]


RuntimeError: ignored

Predict label

In [ ]:
from PIL import Image
import os
import cv2
import numpy as np

# resize成224的檔案路徑
TEST_DATA_PATH = '/content/drive/MyDrive/Colab Notebooks/project/training_224/resized_cut'

img_path = os.path.join(TEST_DATA_PATH)

pics = sorted(os.listdir(img_path))

for i, pic in enumerate(pics):

  img_path2 = os.path.join(img_path, pic)

  img = Image.open(img_path2)

  # Transform
  input = valid_transform(img)

  # unsqueeze batch dimension, in case you are dealing with a single image
  # input = input.unsquueeze(0)

  # Set model to eval
  model.eval()

  input = input.view(1, 3, 224, 224)

  # Get prediction
  output = model(input)

  # vector = np.vectorize(np.int_)
  prediction = int(torch.max(output.data, 1)[1].numpy())+1
  print(f'{pic}={prediction}')
  dict_label[pic] = prediction


Convert to csv

In [ ]:
import csv
# label存成csv的路徑
CSV_PATH = '/content/drive/MyDrive/Colab Notebooks/project/training_224/csv/predict.csv'
# 開啟輸出的 CSV 檔案
with open(CSV_PATH, 'w', encoding = 'utf-8-sig') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)
  writer.writerow(['pic', 'predict_label'])
  for key in dict_label:
  # 寫入一列資料
    writer.writerow([key, dict_label[key]])


In [ ]:
!nvidia-smi